### Callback API Task

In [1]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print("train dataset shape:", train_images.shape, train_labels.shape)
print("test dataset shape:", test_images.shape, test_labels.shape)

train dataset shape: (60000, 28, 28) (60000,)
test dataset shape: (10000, 28, 28) (10000,)


In [2]:
from tensorflow.keras.layers import Layer, Input, Dense, Flatten
from tensorflow.keras.models import Model

INPUT_SIZE = 28

def create_model():
    input_tensor = Input(shape=(INPUT_SIZE, INPUT_SIZE))
    x = Flatten()(input_tensor)
    x = Dense(64, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    return model

In [3]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    return images, targets

def get_preprocessed_ohe(images, targets):
    images, targets = get_preprocessed_data(images, targets)
    oh_targets = to_categorical(targets)

    return images, oh_targets

def get_train_valid_test(train_images, train_targets, test_images, test_targets, validation_size=0.2, random_state=124):
    train_images, train_oh_targets = get_preprocessed_ohe(train_images, train_targets)
    test_images, test_oh_targets = get_preprocessed_ohe(test_images, test_targets)

    train_images, validation_images, train_oh_targets, validation_oh_targets = \
    train_test_split(train_images, train_oh_targets, stratify=train_oh_targets, test_size=validation_size, random_state=random_state)

    return (train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets)

In [5]:
from keras.datasets import mnist

(train_images, train_targets), (test_images, test_targets) = mnist.load_data()

(train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets) = \
get_train_valid_test(train_images, train_targets, test_images, test_targets)

print(train_images.shape, train_oh_targets.shape)
print(validation_images.shape, validation_oh_targets.shape)
print(test_images.shape, test_oh_targets.shape)

(48000, 28, 28) (48000, 10)
(12000, 28, 28) (12000, 10)
(10000, 28, 28) (10000, 10)


In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

# ModelCheckpoint
mcp_cb = ModelCheckpoint(
    filepath = "./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    monitor = 'val_loss',
    save_best_only = False,
    save_weights_only = True,
    mode = 'min'
)

# ReduceLROnPlateau
rlr_cb = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.1,
    patience = 2,
    mode = 'min'
)

# EarlyStopping
ely_cb = EarlyStopping(
    monitor = 'val_loss',
    patience = 3,
    mode = 'min'
)

history = model.fit(x=train_images, y=train_oh_targets, 
                    validation_data=(validation_images, validation_oh_targets), 
                    batch_size=64, epochs=20, callbacks=[mcp_cb, rlr_cb, ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - acc: 0.8230 - loss: 0.6100 - val_acc: 0.9449 - val_loss: 0.1798 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9543 - loss: 0.1568 - val_acc: 0.9553 - val_loss: 0.1453 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9683 - loss: 0.1050 - val_acc: 0.9618 - val_loss: 0.1256 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9749 - loss: 0.0800 - val_acc: 0.9648 - val_loss: 0.1084 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9789 - loss: 0.0661 - val_acc: 0.9719 - val_loss: 0.0909 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9836 - loss: 0.0532 - val_acc: 0.9678 - val_loss: 0.1047 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9854 - loss: 0.0467 - val_acc: 0.9739 - val_loss: 0.0865 - learning_rate: 0.0010
Epoch 

In [8]:
# 저장된 파일 확인
!dir callback_files

 E 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: 6A3C-E039

 E:\kdt_0900_pjw\ai\deep_learning\c_tensorflow\callback_files 디렉터리

2024-05-28 화  오전 11:09    <DIR>          .
2024-05-28 화  오전 11:11    <DIR>          ..
2024-05-28 화  오전 11:09           739,600 weights.001-0.1798-0.9015.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.002-0.1453-0.9571.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.003-0.1256-0.9682.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.004-0.1084-0.9741.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.005-0.0909-0.9791.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.006-0.1047-0.9825.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.007-0.0865-0.9856.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.008-0.0943-0.9868.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.009-0.1112-0.9900.weights.h5
2024-05-28 화  오전 11:09           739,600 weights.010-0.0804-0.9954.weights.h5
2024-05-28 화  오전 11:09   

In [9]:
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step - acc: 0.9749 - loss: 0.0963


[0.08458208292722702, 0.9769999980926514]

In [10]:
# 모델 초기화 하기
model = create_model()
# weight 로드해서 불러오기
model.load_weights('./callback_files/weights.017-0.0776-0.9985.weights.h5')
# model에 컴파일 해주기
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])
# 테스트 확인하기
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 805us/step - acc: 0.9749 - loss: 0.0963


[0.08458208292722702, 0.9769999980926514]